<a href="https://colab.research.google.com/github/marine3005/repository/blob/main/nlp_na_ve_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ADD LIBRAIRIES

In [ ]:
import numpy as np
import pandas as pd
import nltk
import string
import json
from math import log
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('words')
nltk.download('wordnet')
nltk.download('omw-1.4')

stopwords = nltk.corpus.stopwords.words('english')
words = set(nltk.corpus.words.words())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


DOWNLOAD DATA SET

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")
JSON_FILE_PATH="drive/MyDrive/Colab Notebooks/"
JSON_FILE_NAME="News_Category_Dataset_v3.json";
def extractJsonData(jsonData):
  return pd.read_json(jsonData,lines=True);
jsonFile=open(JSON_FILE_PATH+JSON_FILE_NAME);
jsonDataPD=extractJsonData(jsonFile);

Mounted at /content/drive/


In [ ]:
#jsonDataPD.info() #Recuperation of the datatype of the columns of dataframe
#jsonDataPD.isnull() #Check if there is missing values
#jsonDataPD.isnull().sum()#summary of the check is there is missing values
#jsonDataPD.head(10)#affiche les 10 premières lignes de ma dataframe
jsonDataPD['category'].value_counts()#recup the number of value of differents element of a category

POLITICS          35602
WELLNESS          17945
ENTERTAINMENT     17362
TRAVEL             9900
STYLE & BEAUTY     9814
PARENTING          8791
HEALTHY LIVING     6694
QUEER VOICES       6347
FOOD & DRINK       6340
BUSINESS           5992
COMEDY             5400
SPORTS             5077
BLACK VOICES       4583
HOME & LIVING      4320
PARENTS            3955
THE WORLDPOST      3664
WEDDINGS           3653
WOMEN              3572
CRIME              3562
IMPACT             3484
DIVORCE            3426
WORLD NEWS         3299
MEDIA              2944
WEIRD NEWS         2777
GREEN              2622
WORLDPOST          2579
RELIGION           2577
STYLE              2254
SCIENCE            2206
TECH               2104
TASTE              2096
MONEY              1756
ARTS               1509
ENVIRONMENT        1444
FIFTY              1401
GOOD NEWS          1398
U.S. NEWS          1377
ARTS & CULTURE     1339
COLLEGE            1144
LATINO VOICES      1130
CULTURE & ARTS     1074
EDUCATION       

CLEANING OF DATAFRAME


In [ ]:
##erase useless columns
jsonDataPD.drop(['link', 'date', 'authors', 'headline'], axis=1, inplace=True)

#indexNames = jsonDataPD[ (jsonDataPD['category']=="STYLE")&(jsonDataPD['category']=="SCIENCE")&(jsonDataPD['category']=="TECH")&(jsonDataPD['category']=="TASTE")&(jsonDataPD['category']=="MONEY")&(jsonDataPD['category']=="ARTS")&(jsonDataPD['category']=="ENVIRONMENT")&(jsonDataPD['category']=="FIFTY")&(jsonDataPD['category']=="GOOD NEWS")&(jsonDataPD['category']=="U.S NEWS")&(jsonDataPD['category']=="ARTS & CULTURE")&(jsonDataPD['category']=="COLLEGE")&(jsonDataPD['category']=="LATINO VOICES")&(jsonDataPD['category']=="CULTURE & ARTS")&].index
#jsonDataPD.drop(indexNames , inplace=True)

jsonDataPD["category"] = jsonDataPD["category"].replace({"CULTURE & ARTS":"ARTS","ARTS & CULTURE":"ARTS","COLLEGE":"SCIENCE","EDUCATION":"SCIENCE","TECH":"SCIENCE","ENVIRONMENT":"SCIENCE","GREEN":"SCIENCE","DIVORCE":"FAMILY","WEDDINGS":"FAMILY","PARENTS":"FAMILY","PARENTING":"FAMILY","STYLE & BEAUTY":"WOMEN","QUEER VOICES":"VOICES","BLACK VOICES":"VOICES","LATINO VOICES":"VOICES","WORLDPOST":"NEWS","WORLD NEWS":"NEWS","THE WORLDPOST":"NEWS","GOOD NEWS":"NEWS","U.S NEWS":"NEWS","TASTE":"FOOD & DRINK","HOME & LIVING":"HEALTHY LIVING","MONEY":"BUSINESS"})



##enlever la ponctuation
jsonDataPD['short_description']=jsonDataPD['short_description'].str.replace(r'[^\w\s]+', '')

##enlever les majuscules
jsonDataPD['short_description']=jsonDataPD['short_description'].str.lower()

##enlever les chiffres
jsonDataPD['short_description']=jsonDataPD['short_description'].str.replace('\d+', '')

##liste de mots
jsonDataPD['short_description']=jsonDataPD['short_description'].str.lower().str.split()

##enlever mots inutiles
jsonDataPD['short_description']=jsonDataPD['short_description'].apply(lambda x: [item for item in x if item not in stopwords])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: The default value of regex will change from True to False in a future version.


In [ ]:
jsonDataPD['category'].value_counts()#recup the number of value of differents element of a category

POLITICS          35602
FAMILY            19825
WELLNESS          17945
ENTERTAINMENT     17362
WOMEN             13386
VOICES            12060
HEALTHY LIVING    11014
NEWS              10940
SCIENCE           10534
TRAVEL             9900
FOOD & DRINK       8436
BUSINESS           7748
COMEDY             5400
SPORTS             5077
ARTS               3922
CRIME              3562
IMPACT             3484
MEDIA              2944
WEIRD NEWS         2777
RELIGION           2577
STYLE              2254
FIFTY              1401
U.S. NEWS          1377
Name: category, dtype: int64

In [ ]:
from nltk.stem import PorterStemmer
nltk.download('wordnet')
stemmer= PorterStemmer()
sentence=[]
for sentence in jsonDataPD['short_description']:
  for i,word in enumerate(sentence):
     sentence[i] = stemmer.stem(word)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
words = []
sentence=[]
for sentence in jsonDataPD['short_description']:
  for i,word in enumerate(sentence):
    sentence[i] = lemmatizer.lemmatize(word, pos='v')


SPLIT DATAFRAME IN TEST AND TRAIN PART

In [ ]:
# X feature data
X = jsonDataPD['short_description']
# Y is our label
y=jsonDataPD['category']
train,test = train_test_split(jsonDataPD,test_size=0.3)

In [ ]:
train.shape
test.shape

(62859, 2)

Create a dictionary with every words of hitch category and their accurancy

In [ ]:
import copy
dictionary = {}
redondance1 = {}
redondance_inter={}
nb_mot = {}

#from __future__ import division

sentence=[]
keys=train['category'].unique()
for key in keys:
  dictionary[key] =  redondance1
  nb_mot[key] = 0



#PARCOURS de chaque description dataset
for i in range(0, len(train)):
  sentence = train.iloc[i,1]
   #Parcourt de chaque mot de la description
   #Recuperation du dictionnaire de la categorie de la description
  redondance_inter = copy.deepcopy(dictionary[train.iloc[i,0]])
  for mot in sentence:
     #print(dictionary.get(train.iloc[i,0]))
     if (mot in redondance_inter):
     #Si le mot de la description est deja dans le dictionnaire de la categorie de la description
        a = redondance_inter[mot] + 1
        redondance_inter.update({mot:a})
     else :
        redondance_inter.update({mot:1})
     dictionary.update({train.iloc[i,0]:redondance_inter})
     nb_inter = nb_mot[train.iloc[i,0]]
     nb_mot.update({train.iloc[i,0]:nb_inter+1})





Calculate the probability of every words in the different categories

In [ ]:
inter={}
for key2, value in dictionary.items():
  inter = copy.deepcopy(value)
    #Parcourt de tout le dictionnaire redondance et remplacer redondance par proba
  for key1,value1 in inter.items():
    inter.update({key1:(value1/len(inter))})
  dictionary.update({key2:inter})


NAIVE BAYES

In [ ]:
proba_word={}
truepos={}
falseneg={}
falsepos={}
recall={}
precision={}
F1_Score={}

keys=test['category'].unique()
for key in keys:
  truepos[key] =  0
  falseneg[key] = 0
  falsepos[key] = 0
  recall[key]=1
  precision[key]=1
  F1_Score[key]=0


reussite=0
defaite=0
nb_word_total=0
for value in  nb_mot.values() :
  nb_word_total = nb_word_total +value

for i in range(0, len(test)):
  sentence = test.iloc[i,1]
   #Parcourt de chaque mot de la description
   #Recuperation du dictionnaire de la categorie de la description
  for key,value in dictionary.items():
    proba_final=1
    for mot in sentence:
        if mot in value :
          proba_final=proba_final*(value.get(mot))
        else :
          proba_final=proba_final*(1/nb_mot[key])
    proba_word[key]=proba_final * (nb_mot[key]/nb_word_total)

  inter = 0
  #cate_final = "bonjour"
  for key2,value2 in proba_word.items():
    if(value2>inter) :
      inter=value2
      cate_final =key2
  if (cate_final==test.iloc[i,0]) :

      reussite = reussite+1
      c = truepos[cate_final] + 1
      truepos.update({cate_final:c})

  else :

      defaite = defaite+1
      a = falsepos[cate_final] + 1
      b = falseneg[test.iloc[i,0]] + 1
      falsepos.update({cate_final:a})
      falseneg.update({test.iloc[i,0]:b})


for key in recall.keys() :
   d=truepos[key]/(truepos[key]+falseneg[key])
   e=truepos[key]/(truepos[key]+falsepos[key])
   print(d)
   print(e)

   recall.update({key:d})
   print(recall[key])
   precision.update({key:e})
   print(precision[key])
   #F1_Score=(2*precision[key]*recall[key])/(d+e)






0.3877412800541822
0.6711606096131302
0.3877412800541822
0.6711606096131302
0.884811320754717
0.35728162736657654
0.884811320754717
0.35728162736657654
0.676249373224135
0.3527155435445907
0.676249373224135
0.3527155435445907
0.39511823035850496
0.7507246376811594
0.39511823035850496
0.7507246376811594
0.05063291139240506
0.759493670886076
0.05063291139240506
0.759493670886076
0.19516441005802707
0.6020286396181385
0.19516441005802707
0.6020286396181385
0.021568627450980392
0.41509433962264153
0.021568627450980392
0.41509433962264153
0.019342359767891684
0.47619047619047616
0.019342359767891684
0.47619047619047616
0.436919989967394
0.6295627032887604
0.436919989967394
0.6295627032887604
0.13528171135884304
0.6834094368340944
0.13528171135884304
0.6834094368340944
0.1383116883116883
0.7344827586206897
0.1383116883116883
0.7344827586206897
0.07414104882459313
0.4939759036144578
0.07414104882459313
0.4939759036144578
0.8336996336996337
0.3284271284271284
0.8336996336996337
0.3284271284271

In [ ]:
precision

{'TRAVEL': 0.6226880394574599,
 'POLITICS': 0.4167686658506732,
 'FAMILY': 0.3315892160340546,
 'FOOD & DRINK': 0.6643159379407616,
 'ARTS': 0.5855855855855856,
 'ENTERTAINMENT': 0.19247404844290658,
 'IMPACT': 0.2777777777777778,
 'COMEDY': 0.3466666666666667,
 'WOMEN': 0.5060560181680545,
 'NEWS': 0.6515151515151515,
 'SPORTS': 0.5424354243542435,
 'CRIME': 0.33707865168539325,
 'WELLNESS': 0.2381970806822435,
 'HEALTHY LIVING': 0.481994459833795,
 'SCIENCE': 0.5084745762711864,
 'BUSINESS': 0.37644341801385683,
 'VOICES': 0.6762589928057554,
 'FIFTY': 0.0,
 'WEIRD NEWS': 0.2,
 'RELIGION': 0.3020833333333333,
 'MEDIA': 0.36231884057971014,
 'U.S. NEWS': 0.0,
 'STYLE': 0.08379888268156424}

In [ ]:
faux_positif = 0
faux_neg = 0
true_positif =0
for value,key in falsepos.values :
   faux_positif= faux_positif+value

for value in falseneg.values() :
   faux_neg= faux_neg+value

for value in truepos.values() :
   true_positif= true_positif+value

print(faux_positif)
print(faux_neg)
print(true_positif)


TypeError: ignored

In [ ]:
truepos

{'WOMEN': 1699,
 'ENTERTAINMENT': 970,
 'POLITICS': 9308,
 'TRAVEL': 1095,
 'FOOD & DRINK': 1011,
 'WEIRD NEWS': 7,
 'FAMILY': 3947,
 'NEWS': 483,
 'HEALTHY LIVING': 326,
 'COMEDY': 29,
 'VOICES': 394,
 'WELLNESS': 4464,
 'U.S. NEWS': 3,
 'MEDIA': 7,
 'SCIENCE': 330,
 'BUSINESS': 284,
 'IMPACT': 15,
 'FIFTY': 1,
 'STYLE': 7,
 'CRIME': 87,
 'SPORTS': 206,
 'RELIGION': 14,
 'ARTS': 57}

RESULTAT NAIVE BAYES

In [ ]:
test.shape

(62859, 2)

In [ ]:
print(reussite)
print(reussite/(62859)*100)

In [ ]:
print(26053/62859*100)

41.44672998297778


In [ ]:
print(defaite/(62859)*100)

62.89155093144976


In [ ]:
appart_categorie={}


TF-IDF

In [ ]:
dictionary_tf={}
dictionary_tf = copy.deepcopy(dictionary)
inter={}
for key2, value in dictionary_tf.items():
  inter = copy.deepcopy(value)
    #Parcourt de tout le dictionnaire redondance et remplacer redondance par proba
  for key1,value1 in inter.items():
    compteur=0
    for key3, value3 in dictionary.items():
      if key1 in value3 :
        compteur=compteur+1

    inter.update({key1:(value1*(log(28/compteur,10)))})
  dictionary_tf.update({key2:inter})


In [ ]:
proba_word={}
truepos={}
falseneg={}
falsepos={}
recall={}
precision={}
F1_Score={}

keys=test['category'].unique()
for key in keys:
  truepos[key] =  0
  falseneg[key] = 0
  falsepos[key] = 0
  recall[key] = 0
  precision[key]=0
  F1_Score[key]=0



reussite=0
defaite=0
for i in range(0, len(test)):
  sentence = test.iloc[i,1]
   #Parcourt de chaque mot de la description
   #Recuperation du dictionnaire de la categorie de la description
  for key,value in dictionary_tf.items():
    proba_final=1
    for mot in sentence:
        if mot in value :
          proba_final=proba_final+(value.get(mot))
        else :
          proba_final=proba_final+(1/nb_mot[key])
    proba_word[key]=proba_final

  inter = 0
  cate_final = "bonjour"
  for key2,value2 in proba_word.items():
    if(value2>inter) :
      inter=value2
      cate_final =key2
  if (cate_final==test.iloc[i,0]) :

      reussite = reussite+1
      c = truepos[cate_final] + 1
      truepos.update({cate_final:c})

  else :

      defaite = defaite + 1
      a = falsepos[cate_final] + 1
      b = falseneg[test.iloc[i,0]] + 1
      falsepos.update({cate_final:a})
      falseneg.update({test.iloc[i,0]:b})

for key in recall.keys() :
   d=truepos[key]/(truepos[key]+falseneg[key])
   e=truepos[key]/(truepos[key]+falsepos[key])
   recall.update({key:d})
   precision.update({key:e})
   #F1_Score=(2*precision[key]*recall[key])/(precision[key]+recall[key])




In [ ]:
precision

{'TRAVEL': 0.6226880394574599,
 'POLITICS': 0.4167686658506732,
 'FAMILY': 0.3315892160340546,
 'FOOD & DRINK': 0.6643159379407616,
 'ARTS': 0.5855855855855856,
 'ENTERTAINMENT': 0.19247404844290658,
 'IMPACT': 0.2777777777777778,
 'COMEDY': 0.3466666666666667,
 'WOMEN': 0.5060560181680545,
 'NEWS': 0.6515151515151515,
 'SPORTS': 0.5424354243542435,
 'CRIME': 0.33707865168539325,
 'WELLNESS': 0.2381970806822435,
 'HEALTHY LIVING': 0.481994459833795,
 'SCIENCE': 0.5084745762711864,
 'BUSINESS': 0.37644341801385683,
 'VOICES': 0.6762589928057554,
 'FIFTY': 0.0,
 'WEIRD NEWS': 0.2,
 'RELIGION': 0.3020833333333333,
 'MEDIA': 0.36231884057971014,
 'U.S. NEWS': 0.0,
 'STYLE': 0.08379888268156424}

In [ ]:
print(reussite)
print(test.shape)
print(reussite/(62859)*100)

20304
(62859, 2)
32.30086383811388


In [ ]:
faux_positif = 0
faux_neg = 0
true_positif =0
for value in falsepos.values() :
   faux_positif= faux_positif+value

for value in falseneg.values() :
   faux_neg= faux_neg+value

for value in truepos.values() :
   true_positif= true_positif+value



print(faux_positif)
print(faux_neg)
print(true_positif)
